In [116]:
import numpy as np
import pandas as pd

In [91]:
#read in crops
allcrops_2000 = pd.read_csv('datasets/allcrops_2000.csv')
# names of all the crops
crop_names = allcrops_2000['ItemName'].unique()

# remove nan
crop_names = crop_names[:-1]

# list to put our separated crop dataframes
all_crop_dfs = []

# find each crop-separated df and put it in the list of dataframes
for crop in crop_names:
    crop_df = allcrops_2000[(allcrops_2000['ItemName'] == crop)]
    all_crop_dfs.append(crop_df)

In [93]:
# Do the same as above, but for livestock
livestock_2000 = pd.read_csv('datasets/livestock_2000.csv')

# list of livestock names
livestock_names_all = livestock_2000['ItemName'].unique()
livestock_names_all = livestock_names_all[1:-1]
livestock_names = []
for name in livestock_names_all:
    if name != 'Grand Total':
        livestock_names.append(name)

# list to put our separated dataframes
livestock_dfs = []

# separate dataframe based on itemname
for crop in livestock_names:
    crop_df = livestock_2000[(livestock_2000['ItemName'] == crop)]
    livestock_dfs.append(crop_df)

In [94]:
# read in disease rates
column_headers = ["Country", "Year", "Mortality", "Mortality 2", "Mortality 3"]
diabetes = pd.read_csv('datasets/diabetes.csv', names = column_headers)
cardio = pd.read_csv('datasets/cardiovascular.csv', names = column_headers)
cancer= pd.read_csv('datasets/Cancer.csv', names = column_headers)

In [95]:
cardio_2012 = cardio[(cardio["Year"] == "2012")] 
cardio_2000 = cardio[(cardio["Year"] == "2000")] 

In [96]:
diabetes_2012 = diabetes[(diabetes['Year'] == '2012')] 
diabetes_2000 = diabetes[(diabetes['Year'] == '2000')] 

In [97]:
cancer_2012 = cancer[(cancer['Year'] == ' 2012')] 
cancer_2000 = cancer[(cancer['Year'] == ' 2000')] 

In [98]:
all_foods = all_crop_dfs

In [99]:
# dictionary of all dataframes, crop and livestock
all_dfs = {}
for df in all_crop_dfs:
    crop_name = df['ItemName'].values[0]
    all_dfs[crop_name] = df

for df in livestock_dfs:
    crop_name = df['ItemName'].values[0]
    all_dfs[crop_name] = df

In [100]:
for i in range(cancer_2000.shape[0]):
    country = cancer_2000.iloc[i, 0]
    if country == 'United Kingdom of Great Britain and Northern Ireland':
        cancer_2000.iloc[i, 0] = 'United Kingdom'
    elif country == 'China':
        cancer_2000.iloc[i, 0] = 'China, mainland'
    elif country == 'Sudan':
        cancer_2000.iloc[i, 0] = 'Sudan (former)'
    elif country == 'The former Yugoslav republic of Macedonia':
        cancer_2000.iloc[i, 0] = 'The former Yugoslav Republic of Macedonia'
    elif country == 'Cote d\'Ivoire':
        cancer_2000.iloc[i, 0] = 'C\xc3\xb4te d\'Ivoire'
    elif country == 'Czech Republic':
        cancer_2000.iloc[i, 0] = 'Czechia'

/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying t

In [101]:
for i in range(cardio_2000.shape[0]):
    country = cardio_2000.iloc[i, 0]
    if country == 'United Kingdom of Great Britain and Northern Ireland':
        cardio_2000.iloc[i, 0] = 'United Kingdom'
    elif country == 'China':
        cardio_2000.iloc[i, 0] = 'China, mainland'
    elif country == 'Sudan':
        cardio_2000.iloc[i, 0] = 'Sudan (former)'
    elif country == 'The former Yugoslav republic of Macedonia':
        cardio_2000.iloc[i, 0] = 'The former Yugoslav Republic of Macedonia'
    elif country == 'Cote d\'Ivoire':
        cardio_2000.iloc[i, 0] = 'C\xc3\xb4te d\'Ivoire'
    elif country == 'Czech Republic':
        cardio_2000.iloc[i, 0] = 'Czechia'

/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying t

In [102]:
for i in range(diabetes_2000.shape[0]):
    country = diabetes_2000.iloc[i, 0]
    if country == 'United Kingdom of Great Britain and Northern Ireland':
        diabetes_2000.iloc[i, 0] = 'United Kingdom'
    elif country == 'China':
        diabetes_2000.iloc[i, 0] = 'China, mainland'
    elif country == 'Sudan':
        diabetes_2000.iloc[i, 0] = 'Sudan (former)'
    elif country == 'The former Yugoslav republic of Macedonia':
        diabetes_2000.iloc[i, 0] = 'The former Yugoslav Republic of Macedonia'
    elif country == 'Cote d\'Ivoire':
        diabetes_2000.iloc[i, 0] = 'C\xc3\xb4te d\'Ivoire'
    elif country == 'Czech Republic':
        diabetes_2000.iloc[i, 0] = 'Czechia'

/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/angierao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying t

# Looking at missing data

In [103]:
cardio_lands = cardio_2000['Country'].unique()
diabetes_lands = diabetes_2000['Country'].unique()
cancer_lands = cancer_2000['Country'].unique()

In [104]:
df = pd.read_csv('datasets/allwregions_92_00.csv')

In [77]:
all_countries = df['Country'].unique()

### Creates dictionary where each key is a crop whose value is a dictionary with countries as keys and values as the years present in the data

In [29]:
all_crops = df['Item'].unique()
#crop_countries = set()
cropdata = {}
for crop in all_crops:
    data = {}
    for i, country in enumerate(df['Country'].values):
        count = 0
        if country in cancer_lands:
            data[country] = []
    for i, country in enumerate(df['Country'].values):
        if df['Item'].values[i] == crop:
            if country in data:
                if len(data[country]) > 0:
                    data[country].append(df['Year'].values[i])
                else:
                    data[country] = [df['Year'].values[i]]
    cropdata[crop] = data

In [32]:
print len(cropdata['Molasses'].keys())

156


### Next 2 cells: Creates a dictionary with countries as keys and values as the corresponding region

In [33]:
def parse_country(dataset): 
    country_full = pd.read_csv(str(dataset))
    country_array = np.unique(country_full["Country"])
    return country_array

In [106]:
import glob
path = 'datasets/countries/' 
allFiles = glob.glob(path + "/*.csv")
country_dictionary = {}
for file in allFiles:
    area_name = file.split('/')[2].split('.')[0]
    country_list = parse_country(file)
    for country in country_list:
        country_dictionary[country] = area_name

In [107]:
regions = set()
for land in country_dictionary:
    regions.add(country_dictionary[land])

In [108]:
regions_df = pd.read_csv('datasets/onlyregions_92_00.csv')

### Creates a dictionary with the regions as keys and the values as dictionaries with crops as keys and lists of (year, value) tuples as values

In [38]:
regiondata = {}
for i, region in enumerate(regions_df['Country'].unique()):
    regiondata[region] = {}

for i, region in enumerate(regions_df['Country'].values):
    regiondict = regiondata[region]
    crop = regions_df['Item'].values[i]
    year = regions_df['Year'].values[i]
    value = regions_df['Value'].values[i]
    if crop in regiondict:
        regiondata[region][crop].append((year, value))
    else:
        regiondata[region][crop] = [(year, value)]

In [109]:
regions_dfL = pd.read_csv('datasets/onlyregionsL_92_00.csv')

In [110]:
dfL = pd.read_csv('datasets/noregionsL_92_00.csv')

In [111]:
all_ls = dfL['Item'].unique()

### Same thing as cropdata but with livestock

In [112]:
lsdata = {}
for ls in all_ls:
    data = {}
    for i, country in enumerate(dfL['Country'].values):
        count = 0
        if country in cancer_lands:
            data[country] = []
    for i, country in enumerate(dfL['Country'].values):
        if dfL['Item'].values[i] == ls:
            if country in data:
                if len(data[country]) > 0:
                    data[country].append(dfL['Year'].values[i])
                else:
                    data[country] = [dfL['Year'].values[i]]
    lsdata[ls] = data

### Same thing as regiondata but for livestock

In [113]:
regiondataL = {}
for i, region in enumerate(regions_dfL['Country'].unique()):
    regiondataL[region] = {}

for i, region in enumerate(regions_dfL['Country'].values):
    regiondict = regiondataL[region]
    ls = regions_dfL['Item'].values[i]
    year = regions_dfL['Year'].values[i]
    value = regions_dfL['Value'].values[i]
    if ls in regiondict:
        regiondataL[region][ls].append((year, value))
    else:
        regiondataL[region][ls] = [(year, value)]

In [114]:
df = pd.read_csv('datasets/noregions_92_00.csv')

### Functions to extract values from regiondata

In [49]:
def getvalue(region, crop, year):
    if crop in regiondata[region]:
        data = regiondata[region][crop]
        for yr, val in data:
            if yr == year:
                return val
    else:
        return 0

def getvalueL(region, ls, year):
    if ls in regiondataL[region]:
        data = regiondataL[region][ls]
        for yr, val in data:
            if yr == year:
                return val
    else:
        return 0

# Do not re run these cell

### They take awhile to run

### Filling in the data

In [50]:

end = dfL.shape[0]
years = [1992 + i for i in range(9)]
domain = dfL['Domain'].values[0]
element = regions_dfL['Element'].values[0]
unit = dfL['Unit'].values[0]
count = 0
for ls in lsdata:
    for country in lsdata[ls]:
        count += 1
        if len(lsdata[ls][country]) == 9:
            continue
        if len(lsdata[ls][country]) == 8:
            value = getvalueL(country_dictionary[country], ls, 1992)
            dfL.loc[end+1] = [domain, country, element, ls, year, unit, value]
            end += 1
        if len(lsdata[ls][country]) == 1:
            for i in range(1992, 2000):
                value = getvalueL(country_dictionary[country], ls, i)
                dfL.loc[end] = [domain, country, element, ls, i, unit, value]
                end += 1
        if len(lsdata[ls][country]) == 0:
            for i in range(1992, 2001):
                value = getvalueL(country_dictionary[country], ls, i)
                dfL.loc[end] = [domain, country, element, ls, i, unit, value]
                end += 1

77.83031106


In [51]:
end = df.shape[0]
years = [1992 + i for i in range(9)]
domain = df['Domain'].values[0]
element = regions_df['Element'].values[0]
unit = df['Unit'].values[0]
count = 0
for crop in cropdata:
    for country in cropdata[crop]:
        count += 1
        if len(cropdata[crop][country]) == 9:
            continue
        if len(cropdata[crop][country]) == 8:
            value = getvalue(country_dictionary[country], crop, 1992)
            df.loc[end] = [domain, country, element, crop, year, unit, value]
            end += 1
        if len(cropdata[crop][country]) == 1:
            for i in range(1992, 2000):
                value = getvalue(country_dictionary[country], crop, i)
                df.loc[end] = [domain, country, element, crop, i, unit, value]
                end += 1
        if len(cropdata[crop][country]) == 0:
            for i in range(1992, 2001):
                value = getvalue(country_dictionary[country], crop, i)
                df.loc[end] = [domain, country, element, crop, i, unit, value]
                end += 1

827.911561966


### Creating a dictionary with food names as keys and corresponding df as values

In [54]:
crop_names = df['Item'].unique()
all_food_dfs = {}

# find each crop-separated df and put it in the list of dataframes
for crop in crop_names:
    crop_df = df[(df['Item'] == crop)]
    all_food_dfs[crop] = crop_df

In [55]:
ls_names = dfL['Item'].unique()
for ls in ls_names:
    ls_df = dfL[(dfL['Item'] == ls)]
    all_food_dfs[ls] = ls_df

### List of all food names

In [56]:
all_names = list(crop_names) + list(ls_names)

In [58]:
df_vals = {}
for country in df['Country'].unique():
    if country in cancer_lands:
        data = {}
        for crop in all_names:
            df = all_food_dfs[crop]
            for i, name in enumerate(df['Country'].values):
                if name == country:
                    if crop in data:
                        data[crop].append((df['Year'].values[i], df['Value'].values[i]))
                    else:
                        data[crop] = [(df['Year'].values[i], df['Value'].values[i])]
        df_vals[country] = data
                        

In [62]:
predictors = {}
for country in df_vals:
    data = {}
    for crop in df_vals[country]:
        if crop not in df_vals[country]:
            print country, crop
        else:
            year_data = df_vals[country][crop]
            total = 0
            if len(year_data) == 0:
                data[crop] = 0
            else:
                count = 0
                for yr, val in year_data:
                    if not isinstance(val, float) and not isinstance(val, int):
                        continue
                    else:
                        total += val
                        count += 1
                avg = float(total)/float(count)
                data[crop] = avg
    predictors[country] = data
        
        
        
            

In [115]:
x = []
cols = []
for crop in predictors['France']:
    cols.append(crop)
for country in predictors:
    x_vals = []
    for crop in predictors[country]:
        x_vals.append(predictors[country][crop])
    if len(x_vals) != 103:
        print len(x_vals)
        print 'x_vals'
        print len(x_vals)
    x.append(x_vals)

x_df = pd.DataFrame(x, columns = cols)
print x_df.shape

(156, 103)


In [66]:
x_df.to_csv('predictors_filled_156.csv')

In [80]:
cancer = []
for country in predictors:
    for i, place in enumerate(cancer_2000['Country'].values):
        if place == country:
            cancer.append(cancer_2000['Mortality'].values[i])
            
cancer_df = pd.DataFrame(cancer, columns = ['Cancer Mortality Rate'])
print cancer_df.shape
cancer_df.head(n=5)
            

(156, 1)


,Cancer Mortality Rate
0,142.1
1,104.9
2,154.6
3,110.9
4,100.7


In [88]:
cancer_df.to_csv('cancer_156.csv')

In [82]:
diabetes = []
for country in predictors:
    for i, place in enumerate(diabetes_2000['Country'].values):
        if place == country:
            diabetes.append(diabetes_2000['Mortality'].values[i])
            
diabetes_df = pd.DataFrame(diabetes, columns = ['Diabetes Mortality Rate'])
print diabetes_df.shape
diabetes_df.head(n=5)
            

(156, 1)


,Diabetes Mortality Rate
0,14.0
1,19.8
2,5.0
3,15.9
4,28.9


In [87]:
diabetes_df.to_csv('diabetes_156.csv')

In [85]:
cardio = []
for country in predictors:
    for i, place in enumerate(cardio_2000['Country'].values):
        if place == country:
            cardio.append(cardio_2000['Mortality'].values[i])
            
cardio_df = pd.DataFrame(cardio, columns = ['Cardio Mortality Rate'])
print cardio_df.shape
cardio_df.head(n=5)
            

(156, 1)


,Cardio Mortality Rate
0,150.8
1,724.4
2,373.9
3,222.5
4,180.4


In [86]:
cardio_df.to_csv('cardio_156.csv')